<a href="https://colab.research.google.com/github/InbalBolshinsky/LungCancer_Research_Project/blob/main/ExtractFromPDFtoCSV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install pymupdf
import fitz  # PyMuPDF for PDF text extraction
import os
import re
import pandas as pd
import spacy
import openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 59.5 MB/s eta 0:00:00


In [9]:
!pip install spacy scispacy pymupdf pdfplumber
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.4/en_ner_bc5cdr_md-0.5.4.tar.gz


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 93.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 76.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 7.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for en_ner_bc5cdr_md: filename=en_ner_bc5cdr_md-0.5.4-py3-none-any.whl size=119787678 sha256=0dcce8696680bef84376e4ad22c17b354957847bbd310da574ae3dd5abf0fac9
  Stored in directory: /root/.cache/pip/wheels/6e/a6/d6/bd15a41e2ff02a62f0a0a48dddbc07d048307db7199a1538f7
Successfully built en_ner_bc5cdr_md


In [5]:
import spacy
nlp = spacy.load("en_core_sci_sm")  # Load the model


/usr/local/lib/python3.11/dist-packages/spacy/language.py:2195: FutureWarning: Possible set union at position 6328
  deserializers["tokenizer"] = lambda p: self.tokenizer.from_disk(  # type: ignore[union-attr]


In [10]:
# ✅ Load Medical NLP Model
nlp = spacy.load("en_ner_bc5cdr_md")  # Use this instead of en_core_sci_sm

# 🔐 OpenAI API Key (Set this securely, DO NOT hardcode)
openai.api_key = os.getenv("OPENAI_API_KEY")  # Set in Colab secrets or .env file

# 📂 Set up PDF folder (For Google Colab, mount Google Drive)
PDF_FOLDER = "/content/medical_pdfs/"
OUTPUT_CSV = "structured_medical_data.csv"

In [12]:
from google.colab import drive
drive.mount('/content/drive')
PDF_FOLDER = "/content/drive/My Drive/medical_pdfs/"

Mounted at /content/drive


In [11]:
# 🔹 Ensure folder exists in Google Colab (for manual uploads)
if not os.path.exists(PDF_FOLDER):
    os.makedirs(PDF_FOLDER)

In [13]:
def extract_text_from_pdf(pdf_path):
    """Extract raw text from a PDF file using PyMuPDF."""
    try:
        doc = fitz.open(pdf_path)
        text = "\n".join(page.get_text() for page in doc)
        return text
    except Exception as e:
        print(f"❌ Error reading {pdf_path}: {e}")
        return None

def extract_patient_info(text):
    """Extract key medical details from unstructured text using regex & NLP."""

    extracted_data = {
        "Patient Name": None,
        "Age": None,
        "Gender": None,
        "Procedure": None,
        "Diagnosis": None,
        "Medications": None,
        "Doctor's Notes": None
    }

    # 🩺 Use regex to extract structured details
    name_match = re.search(r"(?:Patient Name|Name):\s*(.+)", text, re.IGNORECASE)
    age_match = re.search(r"Age:\s*(\d+)", text, re.IGNORECASE)
    gender_match = re.search(r"Gender:\s*(Male|Female|Other)", text, re.IGNORECASE)
    procedure_match = re.search(r"Procedure:\s*(.+)", text, re.IGNORECASE)
    diagnosis_match = re.search(r"Diagnosis:\s*(.+)", text, re.IGNORECASE)
    medications_match = re.search(r"Medications?:\s*(.+)", text, re.IGNORECASE)
    notes_match = re.search(r"(?:Doctor's Notes|Notes):\s*(.+)", text, re.IGNORECASE)

    if name_match:
        extracted_data["Patient Name"] = name_match.group(1).strip()
    if age_match:
        extracted_data["Age"] = age_match.group(1).strip()
    if gender_match:
        extracted_data["Gender"] = gender_match.group(1).strip()
    if procedure_match:
        extracted_data["Procedure"] = procedure_match.group(1).strip()
    if diagnosis_match:
        extracted_data["Diagnosis"] = diagnosis_match.group(1).strip()
    if medications_match:
        extracted_data["Medications"] = medications_match.group(1).strip()
    if notes_match:
        extracted_data["Doctor's Notes"] = notes_match.group(1).strip()

    return extracted_data

def gpt_extract_medical_info(text):
    """Use GPT to extract structured patient records from unstructured text."""
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "Extract structured medical records from the following text."},
            {"role": "user", "content": text}
        ]
    )

    structured_data = response["choices"][0]["message"]["content"]
    return eval(structured_data)  # Ensure GPT returns a list of dictionaries

def process_all_pdfs(pdf_folder, use_gpt=False):
    """Extract and process data from all PDFs in the folder."""
    all_data = []

    for filename in os.listdir(pdf_folder):
        if filename.endswith(".pdf"):
            pdf_path = os.path.join(pdf_folder, filename)
            print(f"📄 Processing: {pdf_path}")

            # Extract text
            raw_text = extract_text_from_pdf(pdf_path)
            if not raw_text:
                continue

            # Extract structured data
            if use_gpt:
                patient_records = gpt_extract_medical_info(raw_text)
                all_data.extend(patient_records)  # GPT returns multiple records per file
            else:
                patient_info = extract_patient_info(raw_text)
                all_data.append(patient_info)

    # Convert to DataFrame
    df = pd.DataFrame(all_data)
    df.to_csv(OUTPUT_CSV, index=False)
    print(f"✅ CSV saved successfully as {OUTPUT_CSV}")

# Run the pipeline
process_all_pdfs(PDF_FOLDER, use_gpt=False)  # Set use_gpt=True for AI-based structuring

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/My Drive/medical_pdfs/'